In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from IPython.display import FileLink
from tqdm import tqdm_notebook 
from tqdm.notebook import tqdm
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
import warnings
from joblib import Parallel, delayed
from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
train_file = 'training_files/result_{}.csv'
super_train = 'super_train.csv'


In [3]:
stock = 1
df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    
df

,stock,unpredictability_score,Close,High,Low,Open,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,...,Close_trend__benford_correlation,Open_trend__benford_correlation,Open_ trend_upper__benford_correlation,Close_ trend_upper__benford_correlation,Close_trend_lower__benford_correlation,High_lower__benford_correlation,High_trend__benford_correlation,High_upper__benford_correlation,High_trend_lower__benford_correlation,High_ trend_upper__benford_correlation
ID,,,,,,,,,,,,,,,,,,,,,
id_754,1,0,38.4540,38.2343,38.6799,38.4097,1,2017-01-03,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_755,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-04,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_756,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-05,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_757,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-06,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
id_758,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-09,0.062915,0.062915,...,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1503,1,0,NaN,NaN,NaN,NaN,0,2019-12-24,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614
id_1504,1,0,NaN,NaN,NaN,NaN,1,2019-12-26,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614
id_1505,1,0,NaN,NaN,NaN,NaN,0,2019-12-27,-0.064614,-0.064614,...,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614


In [4]:
tdf = pd.read_csv(super_train, index_col='ID')

tdf

,Open_hat,Open_trend,Open_lower,Open_upper,Open_trend_lower,Open_ trend_upper,High_hat,High_trend,High_lower,High_upper,...,Open,High,Low,Close,holiday,unpredictability_score,Open_diff,Low_diff,High_diff,Close_diff
ID,,,,,,,,,,,,,,,,,,,,,
id_0,81.025967,82.386498,78.803918,83.056482,82.386498,82.386498,81.202899,82.597286,79.340480,83.157889,...,82.9961,82.7396,82.9144,82.8101,1,7,1.970133,2.002479,1.536701,1.975953
id_1,80.865875,82.437997,78.822365,82.974286,82.437997,82.437997,81.046504,82.643819,79.061205,83.051594,...,83.1312,83.1669,83.3779,82.9690,0,7,2.265325,2.466027,2.120396,2.151123
id_2,80.976781,82.489497,78.898351,83.082205,82.489497,82.489497,81.080195,82.690352,79.154399,83.025130,...,82.6622,82.7634,82.8984,82.8578,0,7,1.685419,1.803884,1.683205,1.912449
id_3,81.060195,82.540996,78.763372,83.062250,82.540996,82.540996,81.148817,82.736884,79.077222,83.003408,...,83.0279,82.7950,82.8425,82.7385,0,7,1.967705,1.628570,1.646183,1.651943
id_4,81.141042,82.695494,79.101468,83.174879,82.695494,82.695494,81.171321,82.876482,78.954744,83.127788,...,82.3761,82.0828,82.1473,81.8641,0,7,1.235058,0.733757,0.911479,0.672735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_77657,133.246558,130.185331,130.398763,135.963138,128.413669,131.840591,132.715079,129.880891,129.605239,135.501505,...,NaN,NaN,NaN,NaN,0,6,0.153895,0.051313,0.258674,0.197796
id_77658,133.613717,130.157991,130.816426,136.724601,128.277605,131.948423,133.031871,129.850858,129.984751,135.854295,...,NaN,NaN,NaN,NaN,1,6,0.457144,0.291064,0.598355,0.414344
id_77659,133.402365,130.144321,130.475015,136.215723,128.193173,132.004618,132.892882,129.835841,129.924311,135.739193,...,NaN,NaN,NaN,NaN,0,6,0.152253,0.143879,0.246128,0.265842


In [5]:
# print([i for i in df.columns if 'Open' == i])
# print([i for i in df.columns if 'High' == i])
# print([i for i in df.columns if 'Low' == i])
# print([i for i in df.columns if 'holiday' == i])





In [6]:
df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])

In [7]:
cat_cols = [ 
    'holiday',
    'stock',
    'day',
     'month',
     'year',
     'dayofweek',
     'dayofyear',
     'weekofyear',
    'unpredictability_score']
excluded_cols = ['Close_hat', 'Open_hat', 'High_hat', 'Low_hat']
                 

In [8]:
def expand_df(dframe):
    dFrame = dframe.copy()
    dFrame['day'] = dFrame.Date.apply(lambda x: x.day)
    dFrame['month'] = dFrame.Date.apply(lambda x: x.month)
    dFrame['year'] = dFrame.Date.apply(lambda x: x.year)
    dFrame['dayofweek'] = dFrame.Date.apply(lambda x: x.dayofweek)
    dFrame['dayofyear'] = dFrame.Date.apply(lambda x: x.dayofyear)
    dFrame['weekofyear'] = dFrame.Date.apply(lambda x: x.weekofyear)
    dFrame['year_diff'] = dFrame.Date.apply(lambda x: x.year - 2017)
    dFrame['days_so_far_skipped'] = dFrame.Date.apply(lambda x: dFrame[dFrame.Date < x].shape[0])
    dFrame['days_so_far'] = dFrame.Date.apply(lambda x: (x - pd.Timestamp('2017-01-03')).days)
    return dFrame
#     return dFrame[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat'] + ['Date'] + cat_cols + ['year_diff', 'days_so_far_skipped', 'days_so_far'] + ['Close', 'Open', 'High', 'Low'] ]

In [9]:
df = expand_df(df)

In [10]:
# for col in df.columns:
#     if df[col].isna().sum() != 0: print(col)

In [11]:
def rolled_mean(dframe):
    dframe['ID'] = dframe.index

    dFrame = dframe.copy()
    dFrame = roll_time_series(dFrame, show_warnings=False, disable_progressbar=True, column_id='stock', column_sort='Date', max_timeshift=30, min_timeshift=0)
    for col in excluded_cols:
        dframe[col + '_roll_mean'] = dframe['ID'].apply(lambda x: dFrame[dFrame['ID'] == x][col].mean())
        dframe[col + '_roll_max'] = dframe['ID'].apply(lambda x: dFrame[dFrame['ID'] == x][col].max())
        dframe[col + '_roll_min'] = dframe['ID'].apply(lambda x: dFrame[dFrame['ID'] == x][col].min())
        dframe[col + '_roll_range'] = dframe[col + '_roll_max'] - dframe[col + '_roll_min']
        
#     dframe['Close_hat'] = dframe['Close'].fillna(dframe[dframe['Close'].isna()]['Close_hat'])
#     dframe['Open_hat'] = dframe['Open'].fillna(dframe[dframe['Close'].isna()]['Open_hat'])
#     dframe['High_hat'] = dframe['High'].fillna(dframe[dframe['Close'].isna()]['High_hat'])
#     dframe['Low_hat'] = dframe['Low'].fillna(dframe[dframe['Close'].isna()]['Low_hat'])
    
            
    
    return dframe.set_index('ID', drop=True)


In [12]:
df.shape

(754, 525)

In [13]:

df = rolled_mean(df)
df

,stock,unpredictability_score,Close,High,Low,Open,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,...,Open_hat_roll_min,Open_hat_roll_range,High_hat_roll_mean,High_hat_roll_max,High_hat_roll_min,High_hat_roll_range,Low_hat_roll_mean,Low_hat_roll_max,Low_hat_roll_min,Low_hat_roll_range
ID,,,,,,,,,,,,,,,,,,,,,
id_754,1,0,38.4540,38.2343,38.6799,38.4097,1,2017-01-03,0.062915,0.062915,...,38.212551,0.0,38.040274,38.040274,38.040274,0.0,38.501096,38.501096,38.501096,0.0
id_755,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-04,0.062915,0.062915,...,38.261870,0.0,38.079520,38.079520,38.079520,0.0,38.556235,38.556235,38.556235,0.0
id_756,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-05,0.062915,0.062915,...,38.298790,0.0,38.113146,38.113146,38.113146,0.0,38.602518,38.602518,38.602518,0.0
id_757,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-06,0.062915,0.062915,...,38.345896,0.0,38.164707,38.164707,38.164707,0.0,38.636765,38.636765,38.636765,0.0
id_758,1,0,38.5994,38.3790,38.8260,38.5551,0,2017-01-09,0.062915,0.062915,...,38.491169,0.0,38.305092,38.305092,38.305092,0.0,38.781083,38.781083,38.781083,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1503,1,0,NaN,NaN,NaN,NaN,0,2019-12-24,-0.064614,-0.064614,...,44.560687,0.0,44.288012,44.288012,44.288012,0.0,44.765087,44.765087,44.765087,0.0
id_1504,1,0,NaN,NaN,NaN,NaN,1,2019-12-26,-0.064614,-0.064614,...,44.521001,0.0,44.218926,44.218926,44.218926,0.0,44.714463,44.714463,44.714463,0.0
id_1505,1,0,NaN,NaN,NaN,NaN,0,2019-12-27,-0.064614,-0.064614,...,44.537251,0.0,44.249267,44.249267,44.249267,0.0,44.731048,44.731048,44.731048,0.0


In [14]:
# df = rolled_mean(df)
df.shape

(754, 541)

In [15]:
df['Close_hat'].equals(df['Close_hat_roll_mean'])

False

In [16]:
sum(df['Close_hat'] - df['Close_hat_roll_mean'])

6.394884621840902e-14

In [17]:
sum(df[df['Close'].notna()]['Close'] - df[df['Close'].notna()]['Close_hat'])

-0.0021594698716782545

In [18]:
[col for col in tdf.columns.tolist() if col in set(excluded_cols)]

['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']

In [19]:

# ['mean', 'maximum', 'minimum']
df.columns.tolist()

['stock',
 'unpredictability_score',
 'Close',
 'High',
 'Low',
 'Open',
 'holiday',
 'Date',
 'Open_upper__benford_correlation',
 'Open_hat__benford_correlation',
 'Low_hat__benford_correlation',
 'Low_trend__benford_correlation',
 'Low_trend_lower__benford_correlation',
 'Low_ trend_upper__benford_correlation',
 'Close_upper__benford_correlation',
 'Open_hat__fft_coefficient__attr_"abs"__coeff_0',
 'Open_hat__fft_coefficient__attr_"real"__coeff_0',
 'Open_hat__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)',
 'Open_hat__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)',
 'Open_hat__sum_values',
 'Open_hat__quantile__q_0.8',
 'Open_hat__abs_energy',
 'Open_hat__median',
 'Open_hat__mean',
 'Open_hat__quantile__q_0.9',
 'Open_hat__minimum',
 'Open_hat__maximum',
 'Open_hat__quantile__q_0.2',
 'Open_hat__quantile__q_0.3',
 'Open_hat__quant

In [20]:
tdf

,Open_hat,Open_trend,Open_lower,Open_upper,Open_trend_lower,Open_ trend_upper,High_hat,High_trend,High_lower,High_upper,...,Open,High,Low,Close,holiday,unpredictability_score,Open_diff,Low_diff,High_diff,Close_diff
ID,,,,,,,,,,,,,,,,,,,,,
id_0,81.025967,82.386498,78.803918,83.056482,82.386498,82.386498,81.202899,82.597286,79.340480,83.157889,...,82.9961,82.7396,82.9144,82.8101,1,7,1.970133,2.002479,1.536701,1.975953
id_1,80.865875,82.437997,78.822365,82.974286,82.437997,82.437997,81.046504,82.643819,79.061205,83.051594,...,83.1312,83.1669,83.3779,82.9690,0,7,2.265325,2.466027,2.120396,2.151123
id_2,80.976781,82.489497,78.898351,83.082205,82.489497,82.489497,81.080195,82.690352,79.154399,83.025130,...,82.6622,82.7634,82.8984,82.8578,0,7,1.685419,1.803884,1.683205,1.912449
id_3,81.060195,82.540996,78.763372,83.062250,82.540996,82.540996,81.148817,82.736884,79.077222,83.003408,...,83.0279,82.7950,82.8425,82.7385,0,7,1.967705,1.628570,1.646183,1.651943
id_4,81.141042,82.695494,79.101468,83.174879,82.695494,82.695494,81.171321,82.876482,78.954744,83.127788,...,82.3761,82.0828,82.1473,81.8641,0,7,1.235058,0.733757,0.911479,0.672735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_77657,133.246558,130.185331,130.398763,135.963138,128.413669,131.840591,132.715079,129.880891,129.605239,135.501505,...,NaN,NaN,NaN,NaN,0,6,0.153895,0.051313,0.258674,0.197796
id_77658,133.613717,130.157991,130.816426,136.724601,128.277605,131.948423,133.031871,129.850858,129.984751,135.854295,...,NaN,NaN,NaN,NaN,1,6,0.457144,0.291064,0.598355,0.414344
id_77659,133.402365,130.144321,130.475015,136.215723,128.193173,132.004618,132.892882,129.835841,129.924311,135.739193,...,NaN,NaN,NaN,NaN,0,6,0.152253,0.143879,0.246128,0.265842


In [21]:
encoder = LabelEncoder()
for col in tqdm(cat_cols):
    df[col] = encoder.fit_transform(df[col])

In [22]:
# df['train'] = df['Close'].apply(lambda x: not pd.isna(x))
# df.columns = [i.replace('{', '_').replace('}', '_') for i in df.columns]
df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

In [23]:
X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

In [24]:
X['days_so_far']

ID
id_754        0
id_755        1
id_756        2
id_757        3
id_758        6
           ... 
id_1462    1025
id_1463    1028
id_1464    1029
id_1465    1030
id_1466    1031
Name: days_so_far, Length: 713, dtype: int64

In [25]:
X

,stock,unpredictability_score,holiday,Date,Open_upper__benford_correlation,Open_hat__benford_correlation,Low_hat__benford_correlation,Low_trend__benford_correlation,Low_trend_lower__benford_correlation,Low__trend_upper__benford_correlation,...,Open_hat_roll_min,Open_hat_roll_range,High_hat_roll_mean,High_hat_roll_max,High_hat_roll_min,High_hat_roll_range,Low_hat_roll_mean,Low_hat_roll_max,Low_hat_roll_min,Low_hat_roll_range
ID,,,,,,,,,,,,,,,,,,,,,
id_754,0,0,1,2017-01-03,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.212551,0.0,38.040274,38.040274,38.040274,0.0,38.501096,38.501096,38.501096,0.0
id_755,0,0,0,2017-01-04,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.261870,0.0,38.079520,38.079520,38.079520,0.0,38.556235,38.556235,38.556235,0.0
id_756,0,0,0,2017-01-05,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.298790,0.0,38.113146,38.113146,38.113146,0.0,38.602518,38.602518,38.602518,0.0
id_757,0,0,0,2017-01-06,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.345896,0.0,38.164707,38.164707,38.164707,0.0,38.636765,38.636765,38.636765,0.0
id_758,0,0,0,2017-01-09,0.062915,0.062915,0.062915,0.062915,0.062915,0.062915,...,38.491169,0.0,38.305092,38.305092,38.305092,0.0,38.781083,38.781083,38.781083,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_1462,0,0,0,2019-10-25,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.653054,0.0,45.503010,45.503010,45.503010,0.0,45.906257,45.906257,45.906257,0.0
id_1463,0,0,0,2019-10-28,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.602776,0.0,45.420615,45.420615,45.420615,0.0,45.859671,45.859671,45.859671,0.0
id_1464,0,0,0,2019-10-29,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,-0.064614,...,45.583303,0.0,45.424066,45.424066,45.424066,0.0,45.856682,45.856682,45.856682,0.0


In [43]:
X['Low_hat_roll_max'] 

ID
id_754     38.501096
id_755     38.556235
id_756     38.602518
id_757     38.636765
id_758     38.781083
             ...    
id_1462    45.906257
id_1463    45.859671
id_1464    45.856682
id_1465    45.816932
id_1466    45.790248
Name: Low_hat_roll_max, Length: 713, dtype: float64

In [45]:
X['Low_hat_roll_min']

ID
id_754     38.501096
id_755     38.556235
id_756     38.602518
id_757     38.636765
id_758     38.781083
             ...    
id_1462    45.906257
id_1463    45.859671
id_1464    45.856682
id_1465    45.816932
id_1466    45.790248
Name: Low_hat_roll_min, Length: 713, dtype: float64

In [26]:
y

,Close,stock
ID,,
id_754,38.4540,0
id_755,38.5994,0
id_756,38.5994,0
id_757,38.5994,0
id_758,38.5994,0
...,...,...
id_1462,45.9002,0
id_1463,45.9425,0
id_1464,45.9515,0


In [27]:
# hyper_params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'regression',
#     'metric': ['rmse'],
#     'learning_rate': 0.1,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.7,
#     'bagging_freq': 10,
#     'verbose': 0,
#     "max_depth": 4,
#     "num_leaves": 128,  
#     "max_bin": 512,
#     "num_iterations": 100000,
#     "n_estimators": 1000,
#     "random_state": 32
# }
# model_store = [0] * 103
# metrics = [0] * 103
# preds = []
# # for stock in tqdm(X.stock.unique(), total=103):
# for stock in tqdm(range(103), total=103):
# # for stock in tqdm([1]):
#     df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
#     df = expand_df(df)
#     encoder = LabelEncoder()
#     for col in cat_cols:
#         df[col] = encoder.fit_transform(df[col])
        
#     df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]
#     X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
#     X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]
    
    
#     X_tr, X_val, y_tr, y_val = train_test_split(X.drop(columns=['Date']), y['Close'], train_size=0.80, random_state=11568)
    
    
#     model_store[stock] = lgb.LGBMRegressor(**hyper_params)
#     model_store[stock].fit(X_tr, 
#                            y_tr,
#                            eval_set=[(X_val, y_val)],
#                            eval_metric='rmse',
#                            early_stopping_rounds=100,
#                            verbose=False,
#                            )
#     metrics[stock] = list(list(model_store[stock].best_score_.values())[0].values())[0]
#     print(stock, metrics[stock])
#     preds.append(pd.DataFrame({'ID': X_test.index, 'Close': model_store[stock].predict(X_test.drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)}))



# # pd.concat(preds).to_csv('result.csv', index=False)
# # FileLink('result.csv')

In [28]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

In [29]:
model_store1 = [0] * 103
metrics1 = [0] * 103
# df = None

params = {'learning_rate': [.4], 'depth': [2]}


preds1 = []

def get_predictions(stock):
    df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])
    df = expand_df(df)

    df = rolled_mean(df)

    encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = encoder.fit_transform(df[col])

    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

    X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
    X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

    X_tr, X_val, y_tr, y_val = train_test_split(X, y['Close'], train_size=.8, random_state=11568)
    model = CatBoostRegressor(
        loss_function='RMSE', 
        depth=2, 
        learning_rate=0.4, 
        iterations=800,
        random_seed=18,
        od_type='Iter',
        od_wait=20,
        thread_count=1,# task_type="GPU"
    )
    
    model.fit(
        X_tr, y_tr, use_best_model=True,
        cat_features=cat_cols,
        eval_set=(X_val, y_val),
        verbose=False,
        plot=False,
    )
#     grid = GridSearchCV(estimator=model, param_grid=params, cv=2, n_jobs=1, 
#                         scoring=make_scorer(rmse, greater_is_better=False), verbose=0)
#     grid.fit(
#         X, y['Close']
#     )
# #     model_store1[stock] = grid
#     print("best_params")
#     metrics1[stock] = grid.best_params_

    return pd.DataFrame({'Params': str(model.get_feature_importance(prettified=True)[:20]), 'Score': model.best_score_['validation']['RMSE'], 'ID': X_test.index, 'Close': model.predict(X_test)})


num_cores = 7
preds1 = Parallel(n_jobs=num_cores)(delayed(get_predictions)(stock) for stock in tqdm(range(103)))
# preds1 = [get_predictions(stock) for stock in tqdm(range(1))]

In [30]:
preds1[0]['Params'].iloc[0]

'                                           Feature Id  Importances\n0   Close__trend_upper__cwt_coefficients__coeff_0_...    48.190821\n1                                  High_upper__median    12.384971\n2                        High_trend_lower__abs_energy    10.601838\n3                          Low_trend__quantile__q_0_3     6.856522\n4                   Low__trend_upper__quantile__q_0_1     3.704118\n5                                   Open_hat_roll_min     3.164773\n6              Open__trend_upper__benford_correlation     2.136778\n7                   Open_trend_lower__quantile__q_0_3     1.944899\n8                              High_upper__sum_values     1.120060\n9                   High_trend_lower__quantile__q_0_4     1.073567\n10                         Low_lower__quantile__q_0_3     0.808723\n11                  Low__trend_upper__quantile__q_0_4     0.560432\n12  Close_upper__cwt_coefficients__coeff_0__w_10__...     0.521305\n13  Open_upper__fft_coefficient__attr__real___c

In [31]:
model_store = [0] * 103
metrics = [0] * 103

preds = []

parameters = {#'nthread':[1], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.1, .2, .3], #so called `eta` value
              'max_depth': [7, 8, 6],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}


def get_predictions1(stock):
    df = pd.read_csv(train_file.format(stock), index_col='ID', parse_dates=['Date'])
    df = df.join(tdf[['Open_hat', 'High_hat', 'Low_hat', 'Close_hat']])
    df = expand_df(df)

    df = rolled_mean(df)

    encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = encoder.fit_transform(df[col])

    df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

    X, y = df[df['Close'].notna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].notna()]
    X_test, y_test = df[df['Close'].isna()].drop(columns=['Close', 'Open', 'High', 'Low'], axis=1), df[['Close', 'stock']][df['Close'].isna()]

    X_tr, X_val, y_tr, y_val = train_test_split(X, y['Close'], train_size=.8, random_state=11568)
    model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
    
    xgb_grid = GridSearchCV(model,
                           parameters,
                           cv=2,
                           n_jobs=1,
                           scoring=make_scorer(rmse, greater_is_better=False),
                           verbose=False
                           )
    
    xgb_grid.fit(
        X.drop('Date', axis=1), y['Close']
    )
#     model_store[stock] = xgb_grid
#     metrics[stock] = xgb_grid.best_score_
#     print(str(xgb_grid.best_params_))
    return pd.DataFrame({'Params': str(xgb_grid.best_params_), 'Score': abs(xgb_grid.best_score_), 'ID': X_test.index, 'Close': xgb_grid.predict(X_test.drop('Date', axis=1))})


num_cores = 16
# preds = Parallel(n_jobs=num_cores)(delayed(get_predictions1)(stock) for stock in tqdm(range(103)))

# preds = [get_predictions1(stock) for stock in tqdm(range(1))]

In [32]:
# preds[0]

In [33]:
# preds = []
# for stock in X.stock.unique(): 

# # for stock in [1]:
#     pred1 = model_store[stock].predict(X_test[X_test['stock'] == stock].drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)
#     pred2 = model_store1[stock].predict(X_test[X_test['stock'] == stock])
    
#     if metrics[stock] > metrics1[stock]:
#         print("picked first")
# #         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': model_store1[stock].predict(X_test[X_test['stock'] == stock])}))
#         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': (pred2 * 7 + pred1) / 8}))
    
#     else:
#         print("picked second")
# #         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': model_store[stock].predict(X_test[X_test['stock'] == stock].drop(columns=['Date']), num_iteration=model_store[stock].best_iteration_)}))
#         preds.append(pd.DataFrame({'ID': X_test[X_test['stock'] == stock].index, 'Close': (pred2 * 4 + pred1) / 5}))
    

pd.concat(preds1)[['ID', 'Close']].to_csv('result_latest.csv', index=False)

pd.concat(preds1).to_csv('result_catboost_analysis_latest.csv', index=False)

# pd.concat(preds)[['ID', 'Close']].to_csv('result1.csv', index=False)
# pd.concat(preds1).shape       

In [34]:
pd.concat(preds1)

,Params,Score,ID,Close
0,Fea...,1.157025,id_713,118.178473
1,Fea...,1.157025,id_714,117.914669
2,Fea...,1.157025,id_715,118.149437
3,Fea...,1.157025,id_716,118.128691
4,Fea...,1.157025,id_717,117.258484
...,...,...,...,...
36,Fea...,1.308039,id_77657,139.670781
37,Fea...,1.308039,id_77658,138.721452
38,Fea...,1.308039,id_77659,140.792968
39,Fea...,1.308039,id_77660,139.378174


In [35]:
# pd.concat(preds)['Params'].iloc[0]

In [36]:
FileLink('result.csv')

/home/ashutosh/code/AnalyticsVidhya_JANATAHACK/stocks/result.csv

In [37]:
# FileLink('result1.csv')

In [38]:

!cat result.csv

ID,Close
id_713,119.02082681344939
id_714,119.02082681344939
id_715,119.02082681344939
id_716,119.02082681344939
id_717,119.02082681344939
id_718,119.02082681344939
id_719,119.02082681344939
id_720,119.02082681344939
id_721,119.02082681344939
id_722,119.02082681344939
id_723,119.02082681344939
id_724,119.02082681344939
id_725,119.02082681344939
id_726,119.02082681344939
id_727,119.02082681344939
id_728,119.02082681344939
id_729,119.02082681344939
id_730,119.02082681344939
id_731,119.02082681344939
id_732,119.02082681344939
id_733,119.02082681344939
id_734,119.02082681344939
id_735,119.02082681344939
id_736,119.02082681344939
id_737,119.02082681344939
id_738,119.02082681344939
id_739,119.02082681344939
id_740,119.02082681344939
id_741,119.02082681344939
id_742,119.02082681344939
id_743,118.86870666387142
id_744,118.86870666387142
id_745,118.86870666387142
id_746,118.86870666387142
id_747,118.86870666387142
id_748,118.86870666387142
id_749,118.86870666387142
id_750,118.86870666387142
id_

In [39]:
# for stock in X_test.stock.unique():
# !cat result1.csv

In [40]:
!ls | grep csv

final_features.csv
result1.csv
result_catboost_analysis.csv
result_catboost_analysis_latest.csv
result.csv
result_latest.csv
result_xgboost_analysis.csv
SampleSubmission.csv
super_train.csv
Test.csv
Train.csv
